## **1. Data Pre-Processing**

### 1.1. Imports

In [1]:
!pip install transformers datasets scikit-learn torch

In [2]:
import pandas as pd
import random,time,csv
import numpy as np
import math, copy, os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import wandb
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from kaggle_secrets import UserSecretsClient
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE

### 1.2. Adult Census Income Dataset

In [3]:
adult_df = pd.read_csv('/kaggle/input/nlp-project-data-raw/data/adult.data.csv')
adult_df = adult_df.dropna()
adult_df['sex'] = np.where(adult_df['sex'] == ' Male', 1, 0)
adult_df['race'] = np.where(adult_df['race'] != ' White', 0, 1)
adult_df['Probability'] = np.where(adult_df['Probability'] == ' <=50K', 0, 1)

### 1.3. German Credit Dataset

In [4]:
german_df = pd.read_csv('/kaggle/input/nlp-project-data-raw/data/GermanData.csv')
german_df['Probability'] = np.where(german_df['Probability'] == 2, 0,1)
german_df['sex'] = np.where(german_df['sex'] == 'A91', 1, german_df['sex'])
german_df['sex'] = np.where(german_df['sex'] == 'A92', 0, german_df['sex'])
german_df['sex'] = np.where(german_df['sex'] == 'A93', 1, german_df['sex'])
german_df['sex'] = np.where(german_df['sex'] == 'A94', 1, german_df['sex'])
german_df['sex'] = np.where(german_df['sex'] == 'A95', 0, german_df['sex'])

### 1.4. Default Credit Dataset

In [5]:
default_df = pd.read_csv('/kaggle/input/nlp-project-data-raw/data/default_of_credit_card_clients.csv', header=0)
default_df.columns = default_df.iloc[0]
default_df = default_df[1:]
default_df['sex'] = np.where(default_df['SEX'] == '2', 0, 1)

### 1.5. Bank Marketing Dataset

In [6]:
bank_df = pd.read_csv('/kaggle/input/nlp-project-data-raw/data/bank.csv')
bank_df['Probability'] = np.where(bank_df['Probability'] == 'yes', 1, 0)
mean = bank_df.loc[:,"age"].mean()
bank_df['age'] = np.where(bank_df['age'] >= 30, 1, 0)

### 1.6. Medical Expediture Dataset 1

In [7]:
MEPS15 = pd.read_csv('/kaggle/input/medical-expediture-meps-1-2/h181.csv')
MEPS15 = MEPS15.dropna()
MEPS15 = MEPS15.rename(columns = {'FTSTU53X' : 'FTSTU', 'ACTDTY53' : 'ACTDTY', 'HONRDC53' : 'HONRDC', 'RTHLTH53' : 'RTHLTH',
                              'MNHLTH53' : 'MNHLTH', 'CHBRON53' : 'CHBRON', 'JTPAIN53' : 'JTPAIN', 'PREGNT53' : 'PREGNT',
                              'WLKLIM53' : 'WLKLIM', 'ACTLIM53' : 'ACTLIM', 'SOCLIM53' : 'SOCLIM', 'COGLIM53' : 'COGLIM',
                              'EMPST53' : 'EMPST', 'REGION53' : 'REGION', 'MARRY53X' : 'MARRY', 'AGE53X' : 'AGE',
                              'POVCAT15' : 'POVCAT', 'INSCOV15' : 'INSCOV'})
MEPS15 = MEPS15[MEPS15['PANEL'] == 20]
MEPS15 = MEPS15[MEPS15['REGION'] >= 0] # remove values -1
MEPS15 = MEPS15[MEPS15['AGE'] >= 0] # remove values -1
MEPS15 = MEPS15[MEPS15['MARRY'] >= 0] # remove values -1, -7, -8, -9
MEPS15 = MEPS15[MEPS15['ASTHDX'] >= 0] # remove values -1, -7, -8, -9
MEPS15 = MEPS15[(MEPS15[['FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX','EDUCYR','HIDEG',
                             'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                             'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                             'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42','ADSMOK42',
                             'PHQ242','EMPST','POVCAT','INSCOV']] >= -1).all(1)]
MEPS15['RACEV2X'] = np.where((MEPS15['HISPANX'] == 2 ) & (MEPS15['RACEV2X'] == 1), 1, MEPS15['RACEV2X'])
MEPS15['RACEV2X'] = np.where(MEPS15['RACEV2X'] != 1 , 0, MEPS15['RACEV2X'])
MEPS15 = MEPS15.rename(columns={"RACEV2X" : "RACE"})
def utilization(row):
        return row['OBTOTV15'] + row['OPTOTV15'] + row['ERTOT15'] + row['IPNGTD15'] + row['HHTOTD15']
MEPS15['TOTEXP15'] = MEPS15.apply(lambda row: utilization(row), axis=1)
lessE = MEPS15['TOTEXP15'] < 10.0
MEPS15.loc[lessE,'TOTEXP15'] = 0.0
moreE = MEPS15['TOTEXP15'] >= 10.0
MEPS15.loc[moreE,'TOTEXP15'] = 1.0
MEPS15 = MEPS15.rename(columns = {'TOTEXP15' : 'UTILIZATION'})
MEPS15 = MEPS15[['REGION','AGE','SEX','RACE','MARRY',
                                 'FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX',
                                 'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                                 'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                                 'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42', 'ADSMOK42',
                                 'PCS42','MCS42','K6SUM42','PHQ242','EMPST','POVCAT','INSCOV','UTILIZATION', 'PERWT15F']]
MEPS15 = MEPS15.rename(columns={"UTILIZATION": "Probability","RACE" : "race"})

### 1.7. Medical Expediture Dataset 2

In [8]:
MEPS16 = pd.read_csv('/kaggle/input/medical-expediture-meps-1-2/h192.csv')
MEPS16 = MEPS16.dropna()
MEPS16 = MEPS16.rename(columns = {'FTSTU53X' : 'FTSTU', 'ACTDTY53' : 'ACTDTY', 'HONRDC53' : 'HONRDC', 'RTHLTH53' : 'RTHLTH',
                              'MNHLTH53' : 'MNHLTH', 'CHBRON53' : 'CHBRON', 'JTPAIN53' : 'JTPAIN', 'PREGNT53' : 'PREGNT',
                              'WLKLIM53' : 'WLKLIM', 'ACTLIM53' : 'ACTLIM', 'SOCLIM53' : 'SOCLIM', 'COGLIM53' : 'COGLIM',
                              'EMPST53' : 'EMPST', 'REGION53' : 'REGION', 'MARRY53X' : 'MARRY', 'AGE53X' : 'AGE',
                              'POVCAT16' : 'POVCAT', 'INSCOV16' : 'INSCOV'})
MEPS16 = MEPS16[MEPS16['PANEL'] == 21]
MEPS16 = MEPS16[MEPS16['REGION'] >= 0] # remove values -1
MEPS16 = MEPS16[MEPS16['AGE'] >= 0] # remove values -1
MEPS16 = MEPS16[MEPS16['MARRY'] >= 0] # remove values -1, -7, -8, -9
MEPS16 = MEPS16[MEPS16['ASTHDX'] >= 0] # remove values -1, -7, -8, -9
MEPS16 = MEPS16[(MEPS16[['FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX','EDUCYR','HIDEG',
                         'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                         'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                         'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42','ADSMOK42',
                         'PHQ242','EMPST','POVCAT','INSCOV']] >= -1).all(1)]
MEPS16['RACEV2X'] = np.where((MEPS16['HISPANX'] == 2 ) & (MEPS16['RACEV2X'] == 1), 1, MEPS16['RACEV2X'])
MEPS16['RACEV2X'] = np.where(MEPS16['RACEV2X'] != 1 , 0, MEPS16['RACEV2X'])
MEPS16 = MEPS16.rename(columns={"RACEV2X" : "RACE"})
# MEPS16['UTILIZATION'] = np.where(MEPS16['UTILIZATION'] >= 10, 1, 0)

def utilization(row):
        return row['OBTOTV16'] + row['OPTOTV16'] + row['ERTOT16'] + row['IPNGTD16'] + row['HHTOTD16']

MEPS16['TOTEXP16'] = MEPS16.apply(lambda row: utilization(row), axis=1)
lessE = MEPS16['TOTEXP16'] < 10.0
MEPS16.loc[lessE,'TOTEXP16'] = 0.0
moreE = MEPS16['TOTEXP16'] >= 10.0
MEPS16.loc[moreE,'TOTEXP16'] = 1.0

MEPS16 = MEPS16.rename(columns = {'TOTEXP16' : 'UTILIZATION'})

MEPS16 = MEPS16[['REGION','AGE','SEX','RACE','MARRY',
                                 'FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX',
                                 'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                                 'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                                 'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42', 'ADSMOK42',
                                 'PCS42','MCS42','K6SUM42','PHQ242','EMPST','POVCAT','INSCOV','UTILIZATION', 'PERWT16F']]

MEPS16 = MEPS16.rename(columns={"UTILIZATION": "Probability","RACE" : "race"})
protected_attribute = 'race'

## **2. Train-Test Splits for Each Dataset**

In [9]:
# A helper function to split data into train and test sets
def split_data(df, target, test_size=0.3, random_state=42, stratify=True):
    if target not in df.columns:
        raise ValueError(f"Target column '{target}' not found in DataFrame.")
    X = df.drop(columns=[target])
    y = df[target]
    if stratify:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )
    return X_train, X_test, y_train, y_test

# Adult Census Income Dataset
X_train_adult, X_test_adult, y_train_adult, y_test_adult = split_data(adult_df, target="Probability")

# German Credit Dataset
X_train_german, X_test_german, y_train_german, y_test_german = split_data(german_df, target="Probability")

# Default Credit Dataset (ensure target column exists)
default_df.rename(columns={"default payment next month": "Probability"}, inplace=True)
X_train_default, X_test_default, y_train_default, y_test_default = split_data(default_df, target="Probability")

# Bank Marketing Dataset
X_train_bank, X_test_bank, y_train_bank, y_test_bank = split_data(bank_df, target="Probability")

# MEPS15 Dataset
X_train_meps15, X_test_meps15, y_train_meps15, y_test_meps15 = split_data(MEPS15, target="Probability")

# MEPS16 Dataset
X_train_meps16, X_test_meps16, y_train_meps16, y_test_meps16 = split_data(MEPS16, target="Probability")

## **3. Applying SMOTE to Each Dataset**

In [10]:
# Apply SMOTE
smote = SMOTE(random_state=42)

In [11]:
# Note: MEPS15, MEPS16 and Default Credit Datasets did not need encoding for SMOTE as they are already numerical.

# Encoding the X_train and X_test to prepare Adult Dataset for SMOTE
X_train_adult = pd.get_dummies(X_train_adult, drop_first=True)
X_test_adult = pd.get_dummies(X_test_adult, drop_first=True)

# Encoding the X_train and X_test to prepare German Dataset for SMOTE
X_train_german = pd.get_dummies(X_train_german, drop_first=True)
X_test_german = pd.get_dummies(X_test_german, drop_first=True)

# Encoding the X_train and X_test to prepare Bank Dataset for SMOTE
X_train_bank = pd.get_dummies(X_train_bank, drop_first=True)
X_test_bank = pd.get_dummies(X_test_bank, drop_first=True)

# After encoding X_train_adult and X_test_adult, do:
X_test_adult = X_test_adult.reindex(columns=X_train_adult.columns, fill_value=0)

# Similarly, for the German dataset:
X_test_german = X_test_german.reindex(columns=X_train_german.columns, fill_value=0)

# Similarly, for the Bank dataset:
X_test_bank = X_test_bank.reindex(columns=X_train_bank.columns, fill_value=0)

In [12]:
# Adult Census Income Dataset with SMOTE
X_train_adult, y_train_adult = smote.fit_resample(X_train_adult, y_train_adult)

In [13]:
# German Credit Dataset with SMOTE
X_train_german, y_train_german = smote.fit_resample(X_train_german, y_train_german)

In [14]:
# Default Credit Dataset with SMOTE
X_train_default, y_train_default = smote.fit_resample(X_train_default, y_train_default)

In [15]:
# Bank Marketing Dataset with SMOTE
X_train_bank, y_train_bank = smote.fit_resample(X_train_bank, y_train_bank)

In [16]:
# MEPS15 Dataset with SMOTE
X_train_meps15, y_train_meps15 = smote.fit_resample(X_train_meps15, y_train_meps15)

In [17]:
# MEPS16 Dataset with SMOTE
X_train_meps16, y_train_meps16 = smote.fit_resample(X_train_meps16, y_train_meps16)

In [18]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Place all datasets in a dictionary for convenience
datasets = {
    "Adult": (X_train_adult, X_test_adult, y_train_adult, y_test_adult),
    "German": (X_train_german, X_test_german, y_train_german, y_test_german),
    "Default": (X_train_default, X_test_default, y_train_default, y_test_default),
    "Bank": (X_train_bank, X_test_bank, y_train_bank, y_test_bank),
    "MEPS15": (X_train_meps15, X_test_meps15, y_train_meps15, y_test_meps15),
    "MEPS16": (X_train_meps16, X_test_meps16, y_train_meps16, y_test_meps16),
}

In [19]:
def train_and_evaluate_classifiers(X_train, X_test, y_train, y_test, dataset_name):
    print(f"\n{'='*60}")
    print(f"DATASET: {dataset_name.upper()}")
    print(f"{'='*60}")

    # 1. LSR => Logistic Regression
    lsr_model = LogisticRegression(
        max_iter=50,
        random_state=42
    )
    lsr_model.fit(X_train, y_train)
    lsr_preds = lsr_model.predict(X_test)
    lsr_accuracy = accuracy_score(y_test, lsr_preds)
    print(f"\n[LSR - Logistic Regression Results]")
    print(f"Accuracy: {lsr_accuracy:.4f}")
    print(classification_report(y_test, lsr_preds))
    
    # 2. RF => Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=50,  # 50 trees
        random_state=42
    )
    rf_model.fit(X_train, y_train)
    rf_preds = rf_model.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_preds)
    print(f"[RF - Random Forest Results]")
    print(f"Accuracy: {rf_accuracy:.4f}")
    print(classification_report(y_test, rf_preds))

    # 3. SVM => Support Vector Machine
    svm_model = SVC(
        max_iter=50,
        random_state=42
    )
    svm_model.fit(X_train, y_train)
    svm_preds = svm_model.predict(X_test)
    svm_accuracy = accuracy_score(y_test, svm_preds)
    print(f"[SVM - Support Vector Machine Results]")
    print(f"Accuracy: {svm_accuracy:.4f}")
    print(classification_report(y_test, svm_preds))

In [20]:
# Loop over each dataset and train/evaluate all three classifiers
for name, (X_train, X_test, y_train, y_test) in datasets.items():
    train_and_evaluate_classifiers(X_train, X_test, y_train, y_test, dataset_name=name)


DATASET: ADULT

[LSR - Logistic Regression Results]
Accuracy: 0.6576
              precision    recall  f1-score   support

           0       0.83      0.70      0.76     11147
           1       0.36      0.53      0.43      3506

    accuracy                           0.66     14653
   macro avg       0.59      0.61      0.59     14653
weighted avg       0.71      0.66      0.68     14653

[RF - Random Forest Results]
Accuracy: 0.8459
              precision    recall  f1-score   support

           0       0.89      0.91      0.90     11147
           1       0.69      0.64      0.66      3506

    accuracy                           0.85     14653
   macro avg       0.79      0.77      0.78     14653
weighted avg       0.84      0.85      0.84     14653

[SVM - Support Vector Machine Results]
Accuracy: 0.2510
              precision    recall  f1-score   support

           0       0.61      0.04      0.08     11147
           1       0.23      0.91      0.37      3506

    accura